**Aisha Muhammad Nawaz  L200921**  
PySpark Lab 4 - Movies (Using SparkSQL)
8A BSCS MMD  
12th March 2024

Question 1 (35 marks)
You are provided with the dataset “Movies.csv” that contains information about 1600 movies with
properties such as year, length, main actor and actress, director and popularity. Your task is to read the
file in SPARK RDD and efficiently perform the following queries on the dataset using Spark (Scala).
You must use the spark functionality of persisting data in memory and partitioning. Try to write an
efficient query.


In [ ]:
# #Running on Colab
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488493 sha256=409a092c4e59dde6e063b13b5ad0c62d4ece5393e0506b538f00efa2e2617924
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum fonts-ipafont-gothic
  fonts-ipafont-mincho fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 38 not upgraded.
Need to get 39.7 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected package

In [ ]:
#  Import the libraries we will need
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

# Create Spark session and ContextRun PySpark.
# create the session
conf = SparkConf().set("spark.ui.port","4050")
# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.appName("DataFrame").config('spark.ui.port', '4050').getOrCreate()
spark

In [ ]:
df = spark.read.option("delimiter", ";").option("header", "true").option("encoding", "ISO-8859-1").csv('Movies.csv')
# Show the DataFrame
df.show(truncate=False)

+----+------+--------------------------------------+-------+-----------------------+-------------------+------------------------+----------+------+---------------------+
|Year|Length|Title                                 |Genre  |Actor                  |Actress            |Director                |Popularity|Awards|*Image,,,,,          |
+----+------+--------------------------------------+-------+-----------------------+-------------------+------------------------+----------+------+---------------------+
|INT |INT   |STRING                                |CAT    |CAT                    |CAT                |CAT                     |INT       |BOOL  |STRING,,,,,          |
|1990|111   |Tie Me Up! Tie Me Down!               |Comedy |Banderas, Antonio      |Abril, Victoria    |Almodóvar, Pedro        |68        |No    |NicholasCage.png,,   |
|1991|113   |High Heels                            |Comedy |Bosé, Miguel           |Abril, Victoria    |Almodóvar, Pedro        |68        |No    |Nic

In [ ]:
# OR
peopledf =spark.read.csv("Movies.csv",sep= ";",inferSchema=True,header=True)
peopledf.show()

+----+------+--------------------+-------+--------------------+-------------------+--------------------+----------+------+--------------------+
|Year|Length|               Title|  Genre|               Actor|            Actress|            Director|Popularity|Awards|         *Image,,,,,|
+----+------+--------------------+-------+--------------------+-------------------+--------------------+----------+------+--------------------+
| INT|   INT|              STRING|    CAT|                 CAT|                CAT|                 CAT|       INT|  BOOL|         STRING,,,,,|
|1990|   111|Tie Me Up! Tie Me...| Comedy|   Banderas, Antonio|    Abril, Victoria|    Almod�var, Pedro|        68|    No|  NicholasCage.png,,|
|1991|   113|          High Heels| Comedy|        Bos�, Miguel|    Abril, Victoria|    Almod�var, Pedro|        68|    No|  NicholasCage.png,,|
|1983|   104|      Dead Zone, The| Horror| Walken, Christopher|      Adams, Brooke|   Cronenberg, David|        79|    No|   NicholasCag

In [ ]:
# 1. Find the total number of movies of each genre. Consider only the movies released after 1960 and have a length greater than 90 minutes.
# df = spark.read.option("delimiter", ";").option("header", "true").option("encoding", "ISO-8859-1").csv('Movies.csv')
df.filter((df.Year>1960) & (df.Length>90)).groupBy('Genre').count().show()


+---------------+-----+
|          Genre|count|
+---------------+-----+
|        Romance|    1|
|      Adventure|    4|
|           NULL|    1|
|          Drama|  434|
|            War|   15|
|        Fantasy|    1|
|        Mystery|   63|
|          Music|   21|
|Science Fiction|   26|
|         Horror|   26|
|        Western|   37|
|         Comedy|  242|
|         Action|  124|
|       Westerns|    5|
+---------------+-----+



In [ ]:
# 2. For each actress, find the movie in which she acted. You must print the names of the movies and the year of the movie.
df = spark.read.option("delimiter", ";").option("header", "true").option("encoding", "ISO-8859-1").csv('Movies.csv')

result_df = df.groupBy('Actress').agg(
    collect_list('Title').alias('Movies'),
    collect_list('Year').alias('Years')
)

# Show the results
result_df.show(truncate=False)

+----------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# 3. Find the 25 most popular movies released during 1980-1990
df = spark.read.option("delimiter", ";").option("header", "true").option("encoding", "ISO-8859-1").csv('Movies.csv')
df.filter((df.Year>=1980) & (df.Year<=1990)).sort(col('Popularity').desc()).limit(25).collect()

[Row(Year='1986', Length='103', Title='Fire with Fire', Genre='Drama', Actor='Sheffer, Craig', Actress='Madsen, Virginia', Director='Gibbins, Duncan', Popularity='9', Awards='No', *Image,,,,,='NicholasCage.png,,'),
 Row(Year='1989', Length='102', Title='Dead-Bang', Genre='Action', Actor='Johnson, Don', Actress='Miller, Penelope Ann', Director='Frankenheimer, John', Popularity='9', Awards='No', *Image,,,,,='NicholasCage.png,,'),
 Row(Year='1987', Length='104', Title='Lionheart', Genre='Action', Actor='Stoltz, Eric', Actress='Barrymore, Deborah', Director='Schaffner, Franklin J.', Popularity='9', Awards='No', *Image,,,,,='NicholasCage.png,,'),
 Row(Year='1989', Length='104', Title='Tango & Cash', Genre='Action', Actor='Stallone, Sylvester', Actress='Hatcher, Teri', Director='Konchalovsky, Andrei', Popularity='9', Awards='No', *Image,,,,,='NicholasCage.png,,'),
 Row(Year='1989', Length='83', Title='Masque of the Red Death', Genre='Horror', Actor='MacNee, Patrick', Actress='Hoak, Clare', D

In [ ]:
# 4. Find the average length of the movies of each genre.
df = spark.read.option("delimiter", ";").option("header", "true").option("encoding", "ISO-8859-1").csv('Movies.csv')
result_df = df.groupBy('Genre').agg(mean('Length').alias('AverageLength'))

# Show the results
result_df.show(truncate=False)

+---------------+------------------+
|Genre          |AverageLength     |
+---------------+------------------+
|Crime          |66.0              |
|Romance        |127.0             |
|Adventure      |119.0             |
|NULL           |120.5             |
|Drama          |113.30455259026688|
|War            |116.90625         |
|Fantasy        |102.0             |
|Mystery        |103.00990099009901|
|Music          |100.48780487804878|
|Science Fiction|106.47368421052632|
|CAT            |NULL              |
|Horror         |93.92727272727272 |
|Short          |40.0              |
|Western        |93.0091743119266  |
|Comedy         |96.50540540540541 |
|Action         |104.5             |
|Westerns       |124.8             |
+---------------+------------------+



In [ ]:
# 5. Find the lead actor and lead actress pair who have acted in more than one movie together.
df = spark.read.option("delimiter", ";").option("header", "true").option("encoding", "ISO-8859-1").csv('Movies.csv')
df.groupBy('Actor','Actress').agg(count('Title').alias('Count')).filter(col('Count')>1).collect()

[Row(Actor=None, Actress='Madonna', Count=6),
 Row(Actor='Schwarzenegger, Arnold', Actress='Hamilton, Linda', Count=2),
 Row(Actor='Taylor, Robert', Actress=None, Count=3),
 Row(Actor='Grant, Cary', Actress='Bergman, Ingrid', Count=2),
 Row(Actor='Björnstrand, Gunnar', Actress='Andersson, Harriet', Count=3),
 Row(Actor='De Niro, Robert', Actress='Streep, Meryl', Count=2),
 Row(Actor='Pullman, Bill', Actress='Schaal, Wendy', Count=2),
 Row(Actor='Cooper, Gary', Actress=None, Count=2),
 Row(Actor='Van Damme, Jean-Claude', Actress=None, Count=2),
 Row(Actor='Chapman, Graham', Actress=None, Count=6),
 Row(Actor='Trintignant, Jean-Louis', Actress='Aimee, Anouk', Count=3),
 Row(Actor='Cleese, John', Actress='Scales, Prunella', Count=2),
 Row(Actor='Taylor, Rod', Actress=None, Count=3),
 Row(Actor='Nicholson, Jack', Actress='Streep, Meryl', Count=2),
 Row(Actor='Dean, Eddie', Actress=None, Count=2),
 Row(Actor='Depardieu, Gérard', Actress=None, Count=4),
 Row(Actor=None, Actress=None, Count=2

In [ ]:
# 6. Find the names of directors who directed movies of both the mystery and drama genres.
df = spark.read.option("delimiter", ";").option("header", "true").option("encoding", "ISO-8859-1").csv('Movies.csv')
# Filter movies of Mystery genre
A = df.filter(col('Genre') == 'Mystery').select('Director').distinct()

# Filter movies of Drama genre
B = df.filter(col('Genre') == 'Drama').select('Director').distinct()

# Join on the Director column
result_df = A.join(B, 'Director', 'inner')

# Show the results
result_df.show(truncate=False)

+----------------------+
|Director              |
+----------------------+
|Avildsen, John G.     |
|Stern, Steven Hilliard|
|Kurosawa, Akira       |
|Lang, Fritz           |
|Eastwood, Clint       |
|Cimino, Michael       |
|Kubrick, Stanley      |
|Lumet, Sidney         |
|Welles, Orson         |
|Kagan, Jeremy Paul    |
|Locke, Sondra         |
|Lynch, David          |
|Levinson, Barry       |
|Scorsese, Martin      |
|Taylor, Jud           |
|Polanski, Roman       |
|Hitchcock, Alfred     |
|Rafelson, Bob         |
|Mackenzie, John       |
|Pakula, Alan J.       |
+----------------------+
only showing top 20 rows



In [ ]:
# 6 (B). find names of directors and all the genres of movies they have directed
df = spark.read.option("delimiter", ";").option("header", "true").option("encoding", "ISO-8859-1").csv('Movies.csv')
result_df = df.groupBy('Director').agg(collect_set('Genre').alias('Genres'))

# Show the results
result_df.show(truncate=False)

+-----------------------+----------------------------------------------------------------------------------------+
|Director               |Genres                                                                                  |
+-----------------------+----------------------------------------------------------------------------------------+
|NULL                   |[War, Action, Science Fiction, Comedy, Drama, Horror, Music, Westerns, Mystery, Western]|
|Abrahams, Jim          |[Comedy]                                                                                |
|Adolphson, Edvin       |[Comedy]                                                                                |
|Alda, Alan             |[Comedy]                                                                                |
|Aldrich, Robert        |[Action, Comedy, Drama, Mystery, Western]                                               |
|Allegret, Marc         |[Comedy]                                               

In [ ]:
# 7. Find each director and find the average, max, and min ranking of his movies.
df = spark.read.option("delimiter", ";").option("header", "true").option("encoding", "ISO-8859-1").csv('Movies.csv')
df.groupBy('Director').agg(mean('Popularity').alias('Average'),max('Popularity'),min('Popularity')).collect()

[Row(Director=None, Average=40.569169960474305, max(Popularity)='9', min(Popularity)='0'),
 Row(Director='Abrahams, Jim', Average=62.5, max(Popularity)='84', min(Popularity)='41'),
 Row(Director='Adolphson, Edvin', Average=72.0, max(Popularity)='72', min(Popularity)='72'),
 Row(Director='Alda, Alan', Average=32.5, max(Popularity)='53', min(Popularity)='12'),
 Row(Director='Aldrich, Robert', Average=62.2, max(Popularity)='84', min(Popularity)='10'),
 Row(Director='Allegret, Marc', Average=25.0, max(Popularity)='25', min(Popularity)='25'),
 Row(Director='Allen, Craig', Average=None, max(Popularity)=None, min(Popularity)=None),
 Row(Director='Allen, Woody', Average=37.625, max(Popularity)='82', min(Popularity)='14'),
 Row(Director='Almodóvar, Pedro', Average=62.333333333333336, max(Popularity)='73', min(Popularity)='34'),
 Row(Director='Altman, Robert', Average=11.0, max(Popularity)='3', min(Popularity)='19'),
 Row(Director='Amateau, Rod', Average=80.0, max(Popularity)='80', min(Popularit

Question 2
In this example, you will use the given “Movies.csv” data file. Your task is to write an efficient spark code for finding movies (released after 1970) that might be similar or related to a movie made in the 1960’s (1960-1969).
To keep things simple, for comparing two movies we will use the following fields: Genre, director, lead actor, lead actress.
So, we can say two movies might be similar if they have same genre, director and same lead actor or actress.


In [ ]:
df = spark.read.option("delimiter", ";").option("header", "true").option("encoding", "ISO-8859-1").csv('Movies.csv')
A=df.filter(col('Year')>1970)
B=df.filter((col('Year')>=1960) & (col('Year')<=1969))
C=A.join(B,["Genre", "Director", "Actor", "Actress"])
C.show()

+-----+---------------+--------------------+------------+----+------+--------------------+----------+------+------------------+----+------+--------------------+----------+------+------------------+
|Genre|       Director|               Actor|     Actress|Year|Length|               Title|Popularity|Awards|       *Image,,,,,|Year|Length|               Title|Popularity|Awards|       *Image,,,,,|
+-----+---------------+--------------------+------------+----+------+--------------------+----------+------+------------------+----+------+--------------------+----------+------+------------------+
|Drama|Lelouch, Claude|Trintignant, Jean...|Aimee, Anouk|1986|   112|A Man & a Woman: ...|        49|    No|NicholasCage.png,,|1966|   103|Un Hombre y una M...|         6|   Yes|NicholasCage.png,,|
|Drama|Lelouch, Claude|Trintignant, Jean...|Aimee, Anouk|1986|   112|A Man & a Woman: ...|        49|    No|NicholasCage.png,,|1966|   103|     A Man & a Woman|        46|   Yes|NicholasCage.png,,|
+-----+---

In [ ]:
# Stop the Spark session when you're done
spark.stop()